# Ejercicio práctico: Predicción de puntuación. 

**Alumno**: Julián María Galindo Álvarez

**DNI**: 20081281E

# Parte 1: Apartados de 1 a 5.

## Introducción

Este trabajo aborda la tarea de ''Predicción de puntuación'', donde el objetivo va a ser definir una serie de modelos que puedan introducir puntuación en una entrada que carece de puntuación.

En concreto, los signos de puntuación que se van a considerar son:
 * Punto: .
 * Coma: ,
 * Punto y coma: :
 * Dos puntos: :
 * Signo de exclamación: !
 * Signo de interrogación: ?

A lo largo del trabajo se vana  definir dos modelos, uno de puntuación básica y otro basado en 4-gramas. Además se va a definir una serie de funciones para evaluar los modelos usando una función que implementaremos (VerifyPunctuation), basáda en la distancia de Levenshtein.

Este notebook contiene los apartados del 1 al 6 del trabajo, dejando el apartado 7 en en documento TRABAJO_PLN_APARTADO_7_JULIAN_GALINDO_ALVAREZ_20091281E.pdf

Las funciones definidas en cada uno de los apartados se encuentran en los correspondientes archivos python de este directorio. En cualquier caso, este proyecto está disponible en el siguiente repositorio de github:

 * [github.com/julgalalv/PLN_Prediccion_de_puntuacion](https://github.com/julgalalv/PLN_Prediccion_de_puntuacion)
 
La estructura del repositorio es la siguiente:
 * **settings.py**: parámetros globales y creación de directorios en caso necesario. 
 * **preprocessor.py**: En este módulo se definen funciones que son necesarias para el procesamiento de los datos, necesarios para todos los apartados.
 * **punctuator_basic.py**: Funciones del Apartado 1 correspondientes a la puntuación básica.
 * **model_ngram.py**: Clase que define el modelo predictivo basado en N-gramas del Apartado 4.
 * **punctuator_ngram**: Funciones del Apartado 4 correspondientes a la puntuación basada en 4-gramas.
 * **evaluator.py**: Módulo que contiene las funciones para evaluar los módelos usados en el trabajo, esto es, los métodos correspondientes a los Apartados 2, 3 y 5.

En cuanto a los directorios tenemos:

    -root
        |-data  
        |  |-prepared
        |  |-preprocessed
        |  |-raw
        |-predicted
        |-punctuator2tf2
        |-zips

 * **data**: Contiene subdirectorios tando de datos del dataser original como procesados.
 * **raw**: Contiene los corpus de dataset originales.
 * **prepared**: Contiene archivos generados de preprocesamiento de los corpus (apartado 6)
 * **preprocessed**: Contiene archivos generados de preprocesamiento de los corpus vectorizados para el modelo de Ottokar Tilk y Tanel Alum (apartado 6).
 * **predicted**: Contiene archivos con predicciones (puntuaciones) realizadas por los modelos.
 * **punctuator2tf2**: Modelo de Ottokar Tilk y Tanel Alum (apartado 6), implementación en TensorFLow con modificaciones
 realizadas por mí para adaptalo a este trabajo.
 * **zips**: contiene el dataset y el modelo punctuator2tf2 comprimidos.
 
**NOTA**: A lo largo del trabajo voy a ir importando las funciones de cada archivo conforma hagan falta, comentando el código en el propio notebook. En cualquie caso recomiendo ir al código (que está completamente comentado) conforme se avanza en este documento.

En primer lugar se ha definido un módulo **settings** para definir parámetros globales y crear directorios en caso necesario a través de la función *initialize*.

```
def initialize():
    global PUNCT_MARK_DICT      # diccionario con el mapeo de los signos de puntuación y tokens. Usado en el apartado 6
    global PUNCT_MARKS          # lista con signos de puntuación ['.',',',';',':','?','!']
    global MAYUS_MARKS          # lista de signos de puntuación de final de oración ['.','?','!']
    ...
    global CURRENT_DIR              # ruta a directorio raíz
    global DATA_DIR                 # ruta a directorio de datos
    global DATA_RAW_DIR             # ruta a directorio de archivos del dataset 
    global DATA_PREPROCESSED_DIR    # ruta a directorio de archivos de datos preprocesados (con signos tokenizados).
    global DATA_PREPARED_DIR        # ruta a directorio de archivos procesados (Apartado 6)

    global PUNCTUATOR2TF2_DIR      # ruta al directorio de archivos del modelo de Ottokar Tilk y Tanel Alum (Apartado 6)
    global PREDICTED_DIR            # ruta al directorio de archivos con predicciones de modelos
    ...

    PUNCT_MARK_DICT = {".": ".PERIOD", ",": ",COMMA", ";": ";SEMICOLON", ":": ":COLON", "?": "?QUESTIONMARK", 
                       "!":"!EXCLAMATIONMARK"}
    PUNCT_MARKS = ['.',',',';',':','?','!'] # Signos de puntuación (sdp)
    MAYUS_MARKS = ['.','?','!'] # Signos de final de oración
    ...

```

In [1]:
import os
import settings

In [2]:
# Inicializamos parámetros globales y creamos directorios en caso necesario
settings.initialize()

In [3]:
# Definimos variables con las rutas de los archivos de corpus
TEST_RAW_PATH = os.path.join(settings.DATA_RAW_DIR,'PunctuationTask.test.en')
CHECK_RAW_PATH = os.path.join(settings.DATA_RAW_DIR,'PunctuationTask.check.en')
TRAIN_RAW_PATH = os.path.join(settings.DATA_RAW_DIR,'PunctuationTask.train.en')

## APARTADO 1: Modelo de puntuación básica. addPunctuationBasic.

Vamos a implementar un sistema que reciba una expresión como entrada (será una expresión formada solo por minúsculas y sin los signos de puntuación mencionados) y la salida será la misma expresión pero con los cambios correspondientes a la introducción de mayúsculas y signos de puntuación indicados.

Como primera versión de esta función addPunctuationBasic se implementará un modelo que
simplemente cambia la primera letra por mayúscula y añade al final del string de entrada un punto.

```
def addPunctuationBasic(string):
    """
    Función principal del Apartado 1. Pone en mayúscula la primera letra 
    del string  y añade un punto '.' al final si no lo tiene.

    Devuelve el string puntuado.
    """
    initial_upper = change_initial(string,uppercase=True)
    last_char = initial_upper[-1]
    add_dot = '.' if last_char not in settings.PUNCT_MARKS else ''
    return initial_upper + add_dot
```

In [4]:
from punctuator_basic import addPunctuationBasic

Vemos que la función anterior realiza la operación correctamente con el siguiente ejemplo.

In [5]:
addPunctuationBasic('esta es una frase de prueba')

'Esta es una frase de prueba.'

## APARTADO 2: Verificación. VerifyPunctuation

Antes de definir la función verifyPuntuation vamos a definir dos funciones auxiliares que nos servirán.
* **padding(list1,list2)**: Devuelve las los listas de entrada de forma que ambas tengan la misma longitud, añadiendo el elemento string vacío ('') a la lista de menor longitud.

* **tokenizer(text)**: tokeniza el texto de entrada.

```
def tokenizer(text,prepared = False):
    """
    Función que tokeniza un string

    input: 
        text: string a procesar
        prepared: si False, usa la expresión regular  r'?([.|,|;|:|!|?]+) ?' para añadir un espacio
                  a los signos de puntuación para poder ser separados mediante str.split(). En caso contrario el
                  texto ya viene preprocesado (APARTADO 6) y solo es necesario hacer el split(). 

    output: 
        lista de tokens de un string
    """

    if not prepared:
        # Generamos la expresión regular que añade espacios a los signos para poder separarlos con el método split()
        marks = ''.join(settings.PUNCT_MARKS)
        regex1, regex2 = r' ?([{}]+) ?'.format(marks), r' \1 '
        text = re.sub(regex1, regex2, text)
    return text.split()

```

In [6]:
from preprocessor import padding, tokenizer

Veamos el funcionamiento de *padding* con el siguiente ejemplo:

In [7]:
list1, list2 = ['s','a'], ['1','2','3','4']
l1, l2 = padding(list1,list2)
print('Lista 1 con padding: ',l1,'\t Lista 2 con padding: ', l2)
print('Las longitudes son iguales: ', len(l1) == len(l2))

Lista 1 con padding:  ['s', 'a', '', ''] 	 Lista 2 con padding:  ['1', '2', '3', '4']
Las longitudes son iguales:  True


Veamos cómo funciona *tokenizer* con el siguiente ejemplo:

In [8]:
text_example= "Sara said: Hello, what's your name?"
print(tokenizer(text_example))

['Sara', 'said', ':', 'Hello', ',', "what's", 'your', 'name', '?']


Ahora vamos a importar la función principal de este apartado, *verifyPunctuation*. Para definir esta función tokenizamos los strings test y check y hacemos padding para que ambas listas tengan la misma longitud. La idea tras esta función es procesar token a token de izquierda a derecha, comprobando si se ha realizado una eliminación o inserción de manera que se realiza la operación correspondiente en test hasta que ambas listas contienen el mismo número de tokens (no vacíos del padding). Tras esto, se comparan token a token para detectar las sustituciones

```
def verifyPunctuation(check, test, prepared = False):
    """
    Función principal de Apartado 2. Dados un string check y test, devuelve las operaciones
    necesarias para llegar de test a check (con índice de token referente a check). 
    Las operaciones son:
        * (D,i) (Delete): en test se ha eliminado el token i de check.
        * (I,i) (Insertion): en test se ha insertado un token en la posción i de check.
        * (S,i) (Substitution): en test existe un token distinto con respecto al token i de check.

    """
    # Tokenizamos los textos
    check = tokenizer(check,prepared)
    test = tokenizer(test,prepared)
    punct_marks = settings.PUNCT_MARKS if not prepared else settings.PUNCT_MARKS_TOKENS

    # Hacemos padding
    check, test = padding(check, test)
    l_check = len(check)
    modifications = []   
    
    for i in range(l_check):
        # Deletions:
        # Si test[i] no es un sdp pero check[i] si y las palabras anteriores coinciden salvo mayúsculas
        # añadimos la modificación ('D', i) e insertamos el correspondiente sdp faltante en test.
        if check[i] in punct_marks:
            if test[i] not in punct_marks and test[i-1].upper() == check[i - 1].upper():
                modifications.append(('D',i))
                test.insert(i,check[i])
                
        # Reestablecemos el padding para mantener misma longitud
        check, test = padding(check, test)

        # Insertions:
        # Si test[i] es un sdp pero check[i] no y las palabras anteriores coinciden salvo mayúsculas
        # añadimos la modificación ('I', i) y eliminamos el correspondiente sdp en test.
        if check[i] not in punct_marks:
            if test[i] in punct_marks and test[i-1].upper() == check[i - 1].upper():
                modifications.append(('I',i))
                test.pop(i)
                
    # Reestablecemos el padding para mantener misma longitud
    check, test = padding(check, test)
    
    # Substitutions:
    # Tras haber transformado test para añadir los as sustituciones son aquellos elementos que no coinciden
    for i in range(l_check):
        if check[i] != test[i] and check[i] != '' and test[i] != '':
            modifications.append(('S',i))
    return set(modifications)
```

In [9]:
from evaluator import verifyPunctuation

Utilicemos el ejemplo del documento para ver si funciona correctamente la función. Además verificamos que intercambiar check y test devuelve el mismo número de elementos (distancia de Levenshtein).

In [13]:
check_example = "Hello. What's your name?"
test_example = "Hello what's your, name?"
print('check vs test: ', verifyPunctuation(check_example,test_example))
print('test vs check: ', verifyPunctuation(test_example,check_example))

check vs test:  {('D', 1), ('S', 2), ('I', 4), ('S', 5)}
test vs check:  {('S', 1), ('I', 1), ('D', 3), ('S', 5)}


## APARTADO 3: Evaluación de modelos.

Implementaremos una herramienta que permita recorrer todo el corpus de test y verificación. Es decir, irá recorriendo una a una las líneas de cada fichero (que están alineadas), aplicaría sobre la frase de test el algoritmo básico de puntuación (apartado 1: *addPunctuationBasic* ) y a continuación comprobaría si el resultado es o no correcto usando la función *verifyPunctuation* del apartado 2.

En primer lugar vamos a definir una función *evaluate_example* que calcula las métricas precision y recall dado una instancia (check,test). 

La idea tras esta función es usar *verifyPunctuation* para comparar test y check (cambios necesarios), predicho y test (cambios realizados por el modelo) y entre check y predicho (errores y diferencias). A partir de estos cambios podemos calcular la precisión y el recall como:

 * precision: número de modificaciones correctas / número de modificaciones hechas
 * recall: número de modificaciones correctas / número de modificaciones necesarias

En la definición de esta función nos encontramos con el problema de que al basarnos en *verifyPunctuation*, que solo devuelve el tipo de error y su posición, se puede considerar como correcto el haber insertado un signo de puntuación que en el check corresponde a otro (Ej: el modelo añade un '.' y debería haber añadido '?'). Al no tener información sobre el caracter, esto podría dar lugar a un falso positivo. Para solucionar esto añadimos un factor de error basado en comparar si 
en la verificación de la predicción con el check (diferencias) aparecen operaciones distintas, esto es, se ha insertado un signo incorrecto (I) ya que en la verificación aparece una sustitución de ese signo (S).

```
def evaluate_example(punctuationFunction, check, test,model = None, add_punct_basic = False, print_info = True, prepared = False):
    ...
    # Diferencia entre el check y el test original (cambios necesarios)
    vct = verifyPunctuation(test,check,prepared)
    # Diferencia entre el test original y el modificado (modificaciones hechas por la función)
    vtmt = verifyPunctuation(test,punctuated,prepared)
    # Diferencia entre el check y el modificado
    vcmt = verifyPunctuation(punctuated,check,prepared)
    es_correcto = 1 if len(vcmt) == 0 else 0
    # número de cambios hechos por la función            
    hechos_ = len(vtmt)
    # número de cambios necesarios
    necesarios_ = len(vct)
    # número de cambios correctos hechos (intersección entre cambios hechos y necesarios)
    correctos_ = len(vtmt & vct) 
    
    # CORRECCIÓN DE ERROR DE SUSTITUCIÓN DE SIGNO
    # Esta definición de correctos_ es incompleta bajo el uso de verifyPunctuation ya que por ejemplo si
    # el modelo añade al final (token 23 por ejemplo) un '.' y debía añadir '?', en ambos casos recibiremos
    # ('I',23) y sin embargo es incorrecto. En resumen, verifyPunctuation no nos devuelve información sobre el 
    # token en si. Corregimos esto con un factor de error.
    error_ = 0
    check_tokens = tokenizer(check,prepared)
    l_t = len(tokenizer(test,prepared))
    l_c = len(check_tokens)
    cambios = list(vtmt)
    diferencias = list(vcmt)
    # Para cada token que aparece en la verificación de la predicción con el test (cambios), comprobamos si 
    # en la verificación de la predicción con el check (diferencias) aparecen operaciones distintas, esto es,
    # se ha insertado un signo incorrecto (I) ya que en la verificación aparece una sustitución de ese signo (S)
    for j in range(len(cambios)):
        token = cambios[j][1]
        operacion = cambios[j][0]
        for k in range(len(diferencias)):
            error_ += 1 if token == diferencias[k][1] and token<l_t and diferencias[k][1]<l_c and operacion == 'I' and 
            diferencias[k][0] == 'S' else 0
    ...
```

In [14]:
from evaluator import evaluate_example

Comprobamos el funcionamiento de la función con el siguiente ejemplo.

* check = "Hello. What's your name?"
* test = "hello what's your name"

In [15]:
check_example = "Hello. What's your name?"
test_example = "hello what's your name"
evaluate_example(addPunctuationBasic,check_example,test_example,print_info=True)

TEST LINE: 
  hello what's your name
MODEL PUNCTUATED LINE: 
  Hello what's your name.
VALIDATION LINE: 
  Hello. What's your name? 

Modificaciones necesarias:  {('S', 1), ('I', 1), ('I', 4), ('S', 0)}
Modificaciones hechas por el modelo:  {('I', 4), ('S', 0)}
Diferencias entre modelo y validación:  {('S', 1), ('I', 1), ('S', 4)} 

n_hechas (Núm. de modificaciones hechas por el modelo):  2
n_correctas (Núm. de modificaciones correctas, i.e., 
 interseccion(hechas,necesarias) - error de sustitucion de signo):  1
n_necesarias (Núm. de modificaciones necesarias):  4 

precision (n_correctas/n_hechas):  0.5
recall (n_correctas/n_necesarias):  0.25 



(0, 0.5, 0.25, 2, 1, 4)

Notemos que la oración puntuada es "Hello what's your name.", por lo que la función de validación va a considerar como correcto el haber insertado en el token 4, lo que se observa como ('I', 4) tanto en las modificaciones necesarias como las hechas por el modelo. Al no tener información sobre el caracter, esto podría dar lugar a un falso positivo, que es corregido correctamente con el error de sustitución de signo (ver función).

Definamos ahora la función principal de este apartado, *evaluate*, que raliza el proceso anterior para distintos corpus y calcula métricas gobales  (sobre todas las intancias) y métricas medias (medias de las métricas de cada instancia evaluada).

In [16]:
from evaluator import evaluate

Evaluamos la función *addPunctuationBasic*:

In [17]:
evaluate(addPunctuationBasic, check_file_path=CHECK_RAW_PATH, test_file_path=TEST_RAW_PATH)

MÉTRICAS
precision global:  0.9482794650320423
recall global:  0.42824561955393375
F1 global:  0.5900314226893488
precision media:  0.9475733555833681
recall medio:  0.587400667208851
F1 medio:  0.7252308026509773
rendimiento:  0.263384786538729
número de instancias en el corpus:  14382


{'precision_global': 0.9482794650320423,
 'recall_global': 0.42824561955393375,
 'F1_global': 0.5900314226893488,
 'precision_mean': 0.9475733555833681,
 'recall_mean': 0.587400667208851,
 'F1_mean': 0.7252308026509773,
 'score': 0.263384786538729}

Vemos que la precisión es alta, lo que indica que lo que tiene que hacer el modelo (poner la primera letra en mayúscula y un punto al final), lo hace bien. Sin embargo, el recall es relativamente bajo ya que esos cambios no son suficientes para puntuar correctamente las oraciones, cosa que también se deduce del bajo rendimiento.

Aunque no se pide, definimos la función evaluate_example_from_corpus que permite ver la información resultante de evaluate_example para un elemento en concreto del corpus dado el número de línea corpus_line. De esta forma podemos explorar y verificar el correcto funcionamiento de las funciones definidas.

In [18]:
from evaluator import evaluate_example_from_corpus

In [19]:
corpus_line = 2
evaluate_example_from_corpus(addPunctuationBasic,check_file_path=CHECK_RAW_PATH,test_file_path=TEST_RAW_PATH,\
                             corpus_line=corpus_line)

TEST LINE: 
  people working in these canneries could barely stay there all day because of the smell but you know what they came out saying
MODEL PUNCTUATED LINE: 
  People working in these canneries could barely stay there all day because of the smell but you know what they came out saying.
VALIDATION LINE: 
  People working in these canneries could barely stay there all day because of the smell, but you know what they came out saying? 

Modificaciones necesarias:  {('I', 23), ('I', 15), ('S', 0)}
Modificaciones hechas por el modelo:  {('I', 23), ('S', 0)}
Diferencias entre modelo y validación:  {('S', 23), ('I', 15)} 

n_hechas (Núm. de modificaciones hechas por el modelo):  2
n_correctas (Núm. de modificaciones correctas, i.e., 
 interseccion(hechas,necesarias) - error de sustitucion de signo):  1
n_necesarias (Núm. de modificaciones necesarias):  3 

precision (n_correctas/n_hechas):  0.5
recall (n_correctas/n_necesarias):  0.3333333333333333 



## APARTADO 4. Modelo basado en 4-gramas. addPunctuation4gram.

Utilizando el corpus de entrenamiento contenido en PunctuationTask.train.en construimos un modelo de lenguaje inspirado en la idea de 4-gramas.

Definimos una función auxiliar *ngrams* que dado el string text devuelve la lista de N-gramas.

```
def ngrams(text,N):
    """
    Devuelve la lista formada por las N-tuplas de cadenas de tamaño N presentes en el string 'text' (Ngramas)

    input: 
        text: string a procesar
        N: tamaño de las tuplas

    output: 
        lista de Ngramas de un string string
    """
    text = tokenizer(text)
    return [tuple(text[i:i+N]) for i in range(len(text)-N+1)]
```

In [20]:
from preprocessor import ngrams

Veamos un ejemplo:

In [21]:
text_example = 'Sara said: Hello! nice to meet you!'

# Veamos los Ngramas con N de 2 a 4
for n in range(2,5):
    print(str(n)+'-gramas de la oración: ', text_example)
    print('\t', ngrams(text_example,n))

2-gramas de la oración:  Sara said: Hello! nice to meet you!
	 [('Sara', 'said'), ('said', ':'), (':', 'Hello'), ('Hello', '!'), ('!', 'nice'), ('nice', 'to'), ('to', 'meet'), ('meet', 'you'), ('you', '!')]
3-gramas de la oración:  Sara said: Hello! nice to meet you!
	 [('Sara', 'said', ':'), ('said', ':', 'Hello'), (':', 'Hello', '!'), ('Hello', '!', 'nice'), ('!', 'nice', 'to'), ('nice', 'to', 'meet'), ('to', 'meet', 'you'), ('meet', 'you', '!')]
4-gramas de la oración:  Sara said: Hello! nice to meet you!
	 [('Sara', 'said', ':', 'Hello'), ('said', ':', 'Hello', '!'), (':', 'Hello', '!', 'nice'), ('Hello', '!', 'nice', 'to'), ('!', 'nice', 'to', 'meet'), ('nice', 'to', 'meet', 'you'), ('to', 'meet', 'you', '!')]


Creamos la clase **ModelNgram**, que puede generalizarse fácilmente y que instanciaremos con N=4 para definir el modelo propuesto del apartado 4.

Esta clase tiene un método *entrena* para entrenar el modelo a partir de un archivo, un método *predice* que devuelve la operación más probable dada un N-1 tupla. Ademas como atributos cuenta con dos diccionarios:

 * **counts_dict**: para cada operación, cuenta con un diccionario donde cada clave es un N-1 tupla del corpus y su valor es el número de veces que aparece esa tupla precedida de la operación.
 * **trained_dict**: cada clave es una N-1 tupla del corpus de entrenamiento y su valor es la operación más probable (la que cuenta con mayorres repeticiones en count_dict)
 
 ```
     def entrena(self, train_file_path):
        """
        Entrena el modelo dada la ruta al corpus de entrenamiento 'train_file_path'
        """
        # Conjunto de tuplas vistas
        tuplas = set()
        
        # Inicializamos los subdiccionarios de cada operación
        self.counts_dict[self.mayus]= {}
        self.counts_dict[self.minus]={}
        for s in self.punct_marks:
            self.counts_dict[s] = {}
        
        # Recorremos el archivo de entrenamiento
        with open(train_file_path, 'r', encoding='utf-8') as train:
            train_lines = train.readlines()
            for line in train_lines:
                # Eliminamos el espacio final y retorno de carro del final de las líneas
                line = line.rstrip(' \n')
                # Obtenemos los 4gramas de la línea
                ngs = ngrams(line,self.N)
                for ng in ngs:
                    # obtenemos la operación correspondiente a partir del último elemento del 4grama
                    last = ng[-1]
                    op = last if last in self.punct_marks else self.mayus if last.isupper() else self.minus 
                    # Construimos la N-1 tupla  
                    tupla =  tuple([x.lower() for x in list(ng)[:-1]]) if len(ng) > 2 else ng[0]
                    tuplas.add(tupla)
                    # Sumamos una ocurrencia a la entrada correspondiente del diccionario
                    self.counts_dict[op][tupla]= self.counts_dict[op].get(tupla,0) + 1
        train.close()
        # Construimos el diccionario trained_model a partir de counts_dict
        for tupla in tuplas:
            self.trained_dict[tupla] = self.operacion_mas_probable(tupla)
        self.trained = True
 ```

In [22]:
from model_ngram import ModelNgram

Instanciamos el modelo y lo entrenamos.

In [23]:
model4gram = ModelNgram(N=4)
model4gram.entrena(train_file_path=TRAIN_RAW_PATH)

Consultamos el diccionario de conteo para comprobar la distribución de signos dada la terna ('by','the','way')

In [24]:
terna = ('by','the','way')
for i in model4gram.counts_dict:
    print('Ocurrencias con el signo ',i,' para la terna ',terna, ':',model4gram.counts_dict[i].get(terna,0))

Ocurrencias con el signo  <mayus>  para la terna  ('by', 'the', 'way') : 0
Ocurrencias con el signo  <minus>  para la terna  ('by', 'the', 'way') : 25
Ocurrencias con el signo  .  para la terna  ('by', 'the', 'way') : 36
Ocurrencias con el signo  ,  para la terna  ('by', 'the', 'way') : 200
Ocurrencias con el signo  ;  para la terna  ('by', 'the', 'way') : 1
Ocurrencias con el signo  :  para la terna  ('by', 'the', 'way') : 0
Ocurrencias con el signo  ?  para la terna  ('by', 'the', 'way') : 1
Ocurrencias con el signo  !  para la terna  ('by', 'the', 'way') : 0


Se observa que la mayor ocurrencia se da para la coma ',' por lo que la función *predice* del modelo debe devolver dicho caracter.

In [25]:
print('La operación más probable dada la terna ',terna, ' es: ',model4gram.predice(terna))

La operación más probable dada la terna  ('by', 'the', 'way')  es:  ,


Igual que definimos el modelo de forma genérica para Ngramas, vamos a definir la función genérica addPunctuationNgram donde la función requerida por el apartado addPunctuation4gram es addPunctuationNgram usando un modelo basado en 4 gramas.

Esta función cuenta con un parámetro *add_punc_basic* que añade puntuación básica como la de addPuntuationBasic ya que el modelo de 4 gramas nunca va a poner la primera letra en mayúsculas y puede que deje la oración sin puntuación final.

```
def addPunctuationNgram(model,example,add_basic_punct = False):
    ...
    # Comprobamos que el modelo ha sido entrenado
    model.check_entrenado()
    # Trabajamos con los tokens
    tokens = tokenizer(example)
    num_tokens = len(tokens)
    # Generamos los (N-1)-gramas del texto
    N = model.N -1
    grams = ngrams(example,N)
    added_tokens = 0
    for i in range(len(grams)):
        # Calculamos el 4-grama predicho
        operation = model.predice(grams[i]) if N > 1 else model.predice(grams[i][0])
        target_index = i+N+added_tokens
        # Transformamos los tokens del texto
        if operation == model.mayus and target_index < num_tokens:
            tokens[target_index] = change_initial(tokens[target_index], uppercase = True)
        if operation == model.minus and target_index < num_tokens:
            tokens[target_index] = change_initial(tokens[target_index], uppercase = False)
        if operation in model.punct_marks:
            added_tokens += 1
            num_tokens += 1
            tokens.insert(target_index, operation)
            if operation in model.mayus_marks and target_index < num_tokens -1:
                tokens[target_index+1] = change_initial(tokens[target_index+1], uppercase = True)

    # Añadimos los espacios excepto para los signos de puntuación
    result = [' ' + x if x not in model.punct_marks else x for x in tokens]
    # Reconstruimos el texto predicho
    result = ''.join(result)[1:]
    
    # Puesto que el modelo de 4 gramas dificilmente va a poner la primera letra en mayúscula y es probable que
    # deje el final de la oración sin puntuar, añadimos la funcionalidad de addPunctuationBasic como
    # parámetro de la función.
    if add_basic_punct:
        dot = '' if result[-1] in model.punct_marks else '.'
        result = change_initial(result + dot,uppercase=True)
    return result  
```

In [26]:
from punctuator_ngram import addPunctuationNgram

Podemos definir ahora la función de puntuación addPunctuation4gram simplemente como la ejecución de addPunctuationNgram
verificando que el modelo usado usa 4gramas.

In [27]:
def addPunctuation4gram(model,example,add_basic_punct = False):
    N = model.N
    assert N == 4, 'The model is based in {} -grams and it should be 4-grams.'.format(str(N))
    return addPunctuationNgram(model,example,add_basic_punct = add_basic_punct)


In [28]:
text_example = "and we also are eating meat that comes from some of these same places"
print('Frase sin puntuar:\n \t',text_example)
print('Puntuación de la frase con modelo 4 gramas:\n \t',addPunctuation4gram(model4gram,text_example,add_basic_punct=False))
print('Puntuación de la frase con modelo 4 gramas + addPunctuationBasic:\n \t',addPunctuation4gram(model4gram,text_example,add_basic_punct=True))

Frase sin puntuar:
 	 and we also are eating meat that comes from some of these same places
Puntuación de la frase con modelo 4 gramas:
 	 and we also are eating meat, that comes from some of these same places
Puntuación de la frase con modelo 4 gramas + addPunctuationBasic:
 	 And we also are eating meat, that comes from some of these same places.


Exploremos algunos ejemplos usando puntuación básica y sin ella en el modelo de 4gramas.

In [29]:
# Instancia del corpus. Modificar para ver distintos ejemplos.
i = 0

l = 70
print('='*l)
print('MODELO 4GRAMS')
print('='*l)
evaluate_example_from_corpus(addPunctuation4gram, model = model4gram,test_file_path=TEST_RAW_PATH,\
                             check_file_path= CHECK_RAW_PATH, add_punct_basic=False,corpus_line = i)
print('='*l)
print('='*l)
print('4GRAMS + PUNTUACION BÁSICA')
print('='*l)
evaluate_example_from_corpus(addPunctuation4gram, model = model4gram, test_file_path=TEST_RAW_PATH,\
                             check_file_path= CHECK_RAW_PATH,add_punct_basic=True,corpus_line = i)
print('='*l)

MODELO 4GRAMS
TEST LINE: 
  it can be a very complicated thing the ocean
MODEL PUNCTUATED LINE: 
  it can be a very complicated thing, the ocean
VALIDATION LINE: 
  It can be a very complicated thing, the ocean. 

Modificaciones necesarias:  {('I', 7), ('I', 9), ('S', 0)}
Modificaciones hechas por el modelo:  {('I', 7)}
Diferencias entre modelo y validación:  {('I', 10), ('S', 0)} 

n_hechas (Núm. de modificaciones hechas por el modelo):  1
n_correctas (Núm. de modificaciones correctas, i.e., 
 interseccion(hechas,necesarias) - error de sustitucion de signo):  1
n_necesarias (Núm. de modificaciones necesarias):  3 

precision (n_correctas/n_hechas):  1.0
recall (n_correctas/n_necesarias):  0.3333333333333333 

4GRAMS + PUNTUACION BÁSICA
TEST LINE: 
  it can be a very complicated thing the ocean
MODEL PUNCTUATED LINE: 
  It can be a very complicated thing, the ocean.
VALIDATION LINE: 
  It can be a very complicated thing, the ocean. 

Modificaciones necesarias:  {('I', 7), ('I', 9), ('S

## APARTADO 5: Evaluación de addPunctuation4gram y comparación de modelos.

Evaluemos el modelo usando la misma función *evaluate* anterior. Vamos a comparar el rendimiento usando también la puntuación básica y sin ella.

In [30]:
print('MODELO 4GRAMS')
evaluate(addPunctuation4gram,model=model4gram,test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH)
print()

MODELO 4GRAMS
MÉTRICAS
precision global:  0.3061919504643963
recall global:  0.04666708609896505
F1 global:  0.08099033684555332
precision media:  0.14064391487397296
recall medio:  0.04956605113889393
F1 medio:  0.07329966587077798
rendimiento:  0.0
número de instancias en el corpus:  14382



Podemos ver que este modelo tal cual es muy pobre. Usando la función *evaluate_example_from_corpus*, podemos ver que nunca pone la mayúscula inicial como era de esperar y rara vez un punto al final. Además las predicciones parecen bastante pobres y esto se puede deber a la falta de variedad en en las tuplas del corpus de entrenamiento. Una posible mejoría se conseguiría con un corpus más grande y variado.

Veamos la evaluación añadiendo la puntuación básica y lo comparamos con lo obtenido en addPunctuationBasic.

In [31]:
print('4GRAMS + PUNTUACION BÁSICA')
evaluate(addPunctuation4gram,model=model4gram,test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH,\
         add_punct_basic=True)
print()
print('PUNTUACIÓN BÁSICA')
evaluate(addPunctuationBasic,test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH,)
print()

4GRAMS + PUNTUACION BÁSICA
MÉTRICAS
precision global:  0.7801183367117734
recall global:  0.447937965963069
F1 global:  0.569101954358339
precision media:  0.8433161902915692
recall medio:  0.5991575233871246
F1 medio:  0.7005732377871406
rendimiento:  0.2369628702544848
número de instancias en el corpus:  14382

PUNTUACIÓN BÁSICA
MÉTRICAS
precision global:  0.9482794650320423
recall global:  0.42824561955393375
F1 global:  0.5900314226893488
precision media:  0.9475733555833681
recall medio:  0.587400667208851
F1 medio:  0.7252308026509773
rendimiento:  0.263384786538729
número de instancias en el corpus:  14382



Podemos ver que en general la precisión es más baja en el modelo de 4gramas pero el recall ligeramente superior, pero no de forma sustancial. Además los F1 son menores en el modelo de puntuación básica que en el de 4 gramas así como el rendimiento.

En general podemos concluir que el modelo de puntuación básica es ligeramente mejor que el basado en 4gramas cuando se añade la puntuación línea a línea por sorprendente que parezca. Esto puede deverse a que muchas de las oraciones del corpus de entrenamiento no presentan signos intermedios (sólo mayúscula inicial y punto final, exactamente lo que hace addPunctuationBasic)

Ya que hemos implementado un modelo genérico, vamos a comparar con modelos basados en 3gramas y 5gramas para ver si existe mejoría o no.

In [32]:
model3gram = ModelNgram(N=3)
model5gram = ModelNgram(N=5)
model3gram.entrena(train_file_path=TRAIN_RAW_PATH)
model5gram.entrena(train_file_path=TRAIN_RAW_PATH)

In [33]:
print('3GRAMAS')
evaluate(addPunctuationNgram, model = model3gram,test_file_path=TEST_RAW_PATH,\
         check_file_path= CHECK_RAW_PATH, add_punct_basic=True)

3GRAMAS
MÉTRICAS
precision global:  0.7269191453289785
recall global:  0.45858630343829626
F1 global:  0.5623848698486792
precision media:  0.8041213477542051
recall medio:  0.6055814495149251
F1 medio:  0.6908704051694091
rendimiento:  0.2259769155889306
número de instancias en el corpus:  14382


{'precision_global': 0.7269191453289785,
 'recall_global': 0.45858630343829626,
 'F1_global': 0.5623848698486792,
 'precision_mean': 0.8041213477542051,
 'recall_mean': 0.6055814495149251,
 'F1_mean': 0.6908704051694091,
 'score': 0.2259769155889306}

In [34]:
print('5GRAMAS')
evaluate(addPunctuationNgram, model = model5gram, test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH,add_punct_basic=True)
print()

5GRAMAS
MÉTRICAS
precision global:  0.8715371568565865
recall global:  0.43644027808361385
F1 global:  0.581621530980129
precision media:  0.9022099935324751
recall medio:  0.592608753166226
F1 medio:  0.7153476507331434
rendimiento:  0.2533027395355305
número de instancias en el corpus:  14382



Observamos que en cuanto a precision, el modelo basado en 3gramas es ligeramente inferior al de 4gramas y este a su vez inferior al de 5 gramas, todos por debajo de la precisión del puntuador básico. En cuanto a recall el orden es inverso: el modelo basado en 3 gramas presenta mayor recall que el de 4 y este a su vez que el de 5.
En cuanto al F1, los mayores valores los encontramos para el modelo de 5gramas y de puntuación básica. Es obvio que cuanto mayor sea N, menos probable es la probabilidad de que se de una N-tupla concreta por lo que el modelo basado en N-gramas con N grande coincidirá eventualmente con el de puntuación básica si se considera el parámetro add_punct_basic.

Para acabar este apartado podemos explorar algunas predicciones de los modelos de 3 y 5 gramas:

In [35]:
# Instancia del corpus. Modificar para ver distintos ejemplos.
i= 0

l = 70 
print('='*l)
print('MODELO 3GRAMS')
print('='*l)
evaluate_example_from_corpus(addPunctuationNgram, model = model3gram,test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH, add_punct_basic=True,corpus_line = i)
print('='*l)
print('MODELO 5GRAMS')
print('='*l)
evaluate_example_from_corpus(addPunctuationNgram, model = model5gram,test_file_path=TEST_RAW_PATH, check_file_path= CHECK_RAW_PATH, add_punct_basic=True,corpus_line = i)

MODELO 3GRAMS
TEST LINE: 
  it can be a very complicated thing the ocean
MODEL PUNCTUATED LINE: 
  It can be a very complicated thing. The ocean.
VALIDATION LINE: 
  It can be a very complicated thing, the ocean. 

Modificaciones necesarias:  {('I', 7), ('I', 9), ('S', 0)}
Modificaciones hechas por el modelo:  {('S', 7), ('I', 7), ('I', 9), ('S', 0)}
Diferencias entre modelo y validación:  {('S', 7), ('S', 8)} 

n_hechas (Núm. de modificaciones hechas por el modelo):  4
n_correctas (Núm. de modificaciones correctas, i.e., 
 interseccion(hechas,necesarias) - error de sustitucion de signo):  2
n_necesarias (Núm. de modificaciones necesarias):  3 

precision (n_correctas/n_hechas):  0.5
recall (n_correctas/n_necesarias):  0.6666666666666666 

MODELO 5GRAMS
TEST LINE: 
  it can be a very complicated thing the ocean
MODEL PUNCTUATED LINE: 
  It can be a very complicated thing, the ocean.
VALIDATION LINE: 
  It can be a very complicated thing, the ocean. 

Modificaciones necesarias:  {('I', 

# Parte 2. Apartado 6.

## Introducción

Utilizando también ejemplos de TED talks, en un artículo de 2016, Ottokar Tilk y Tanel Alum ha aplicado un modelo de redes recurrentes bidireccionales para esta misma tarea (restauración de signos de puntuación en textos no segmentados).
El artículo donde lo describen se encuentra publicado en este enlace:
 * [Bidirectional Recurrent Neural Network with Attention Mechanism for Punctuation Restoration (InterSpeech 2016)](https://www.isca-speech.org/archive_v0/Interspeech_2016/pdfs/1517.PDF)

El código correspondiente a su implementación se encuentra también disponible en github:
 * https://github.com/ottokart/punctuator2

En este apartado vamos a estudiar la implementación de los autores y adaptar su trabajo al escenario de este ejercicio, entrenando el modelo y realizando la puntuacion del archivo PunctuationTask.test.en, para luego evaluarlo usando el método tanto de los autores como la función *evaluate* implementada en el la parte 1.

## Modelo

Los autores presentan un modelo de red neuronal recurrente bidireccional con mecanismo de atención para la restauración de la puntuación en textos no segmentados. El modelo puede utilizar contextos largos en ambas direcciones y dirigir la atención cuando sea necesario, lo que le permite superar el estado del arte anterior en los conjuntos de datos de inglés (IWSLT2011) y estonio por un amplio margen.

El modelo es una red neuronal recurrente bidireccional (BRNN) que le permite hacer uso de contextos de longitud no fija antes y después de la posición actual en el texto. En las capas recurrentes se utilizan unidades recurrentes cerradas (GRU) que son muy adecuadas para capturar dependencias de largo alcance en múltiples escalas de tiempo. Estas unidades tienen beneficios similares a los de las unidades LSTM, pero son más simples. Incorporan un mecanismo de atención  al modelo para aumentar su capacidad de encontrar partes relevantes del contexto para las decisiones de puntuación. Por ejemplo, el modelo podría centrarse en palabras que indican una pregunta, pero que pueden estar relativamente lejos de la palabra actual, para empujar al modelo a terminar la frase con un signo de interrogación en lugar de un punto. 

En la implementación , los signos de puntuación se encuentran tokenizados, esto es, aparecen como:
   * "." -> .PERIOD
   * "," -> ,COMMA
   * ";" -> ;SEMICOLON
   * ":" -> :COLON
   * "?" -> ?QUESTIONMARK
   * "!" -> !EXCLAMATIONMARK

Sin embargo, el input del modelo requiere de un embedding de texto. Para ello, se disponde de un módulo data.py que vectoriza textos preparados con la tokenización anterior extrayendo el vocabulario de un corpues de entrenamiento.

## Implementación

El modelo que vamos a usar aquí no es exactamente el del repositorio anterior, que usa Theano y es muy lento a la hora de entrenar y puntuar. Para ello usamos la implementacion en TensorFlow dada por los propios autores en  el siguiente repositorio:

 * https://github.com/cadia-lvl/punctuation-prediction/tree/master/punctuator2tf2

Este modelo se encuentra en el directorio **punctuator2tf2** y sus principales módulos son los siguientes:
 
 * **data.py**: vectoriza textos preparados tokenización extrayendo el vocabulario de un corpues de entrenamiento.
 * **models.py**: definición del modelo.
 * **main.py**: entrena el modelo y genera un archivo con los parámetros del mismo.
 * **punctuator.py**: puntúa un archivo de texto dado un modelo preentrenado.
 * **error_calculator.py**: evalúa untexto puntuado a partir de otro de validación calculando la precisión, recall y F1.

## Procesos y modificaciones

 El proceso que vamos a ver en este documento consta ded las siguientes fases:

  1. Preprocesamiento de los datos crudos: usamos funciones que hemos definido en el módulo **preprocessor.py** para transformar los signos en los tokens anteriores, asi como los numéricos por el token "<NUM\>"
  1. Generación de archivos preparados: usamod el módulo **data.py** para vectorizar los textos preprocesados.
  1. Entrenamos un modelo de 256 capas con learning rate de 0.02 durante 8 épocas con tamaño de minibatch 64.
  1. Puntuamos el archivo PunctuationTask.test.en.
  1. Evaluamos la puntuación con el módulo **error_calculator.py** de los autores.
  1. Evaluamos la puntuación con nuestro módulo **evaluator** basado en *VerrifyPunctuation* definido en la parte 1.

Para llevar esto a cabo y mejorar el propio modelo de los autores se ha realizado una modificación en el módulo **punctuator.py**, ya que el modelo original no pone en mayúsculas las palabras tras signos de final de oración. Además se ha añadido puntuación básica (primera letra en mayúsculas y punto al final si no hay un signo de final de oración), pues el modelo suele dejar los finales de oraciones sin puntuar con con signos intermedios (Ej.: comas). 

Fragmento modificado en **punctuator.py**:
```
def restore(text, word_vocabulary, reverse_punctuation_vocabulary, model):
    ...
    # iteramos en los elementos de la secuencia predicha
    for j in range(step):
       # si no es el final
       if j < step - 1:
           # añadimos el signo de puntuación correspondiente si no es un espacio
           punctuated += " " + punctuations[j] + " " if punctuations[j] != data.SPACE else " "
           # AÑADIDO POR JULIÁN: si el signo añadido era EOS, ponemos en mayúsculas la siguiente palabra.
           punctuated += subsequence[1+j] if not punctuations[j] in data.EOS_TOKENS  else to_upper(subsequence[1+j])
        # AÑADIDO POR JULIÁN: si el último token no es EOS se añade el token de punto (PUNTUACIÓN BÁSICA)
        elif j == step -1: punctuated += " .PERIOD " if punctuations[j] not in data.EOS_TOKENS else  " " + 
          punctuations[j] + " "
    if subsequence[-1] == data.END:
        break
    i += step
  # AÑADIDO POR JULIÁN: Se pone en mayúsculas la primera letra (PUNTUACIÓN BÁSICA)
  return to_upper(punctuated)

```

**IMPORTANTE:**
 * Esta parte del notebook está pensada para ser ejecutado a parte en Google Collab con GPU ya que el modelo tarda mucho en entrenar y puntuar (entorno a 1 hora). Por ello se van a comentar las celdas referentes a esos procesos en caso de que se ejecuten sin querer.

**NOTA**: Al igual que en la parte 1, iré cargando las funciones implementadas conforme se vayan explicando, por lo que recomiendo consultar el código de los módulos.

En caso de que se quieran ejecutar las distintas celdas, es posible que haya que instalar las dependencias definidas por los autores. Para ello se ejecuta la siguiente celda comentada.

In [3]:
#!pip install -r {settings.PUNCTUATOR2TFT_DIR}/requirements.txt

Para preparar los archivos (sustituir los signos por tokens), utilizamos la función *prepare_file* definida en **preprocessor**, que realiza transformaciones como la del siguiente ejemplo:

 * Sin procesar: Yeah, there's 38 million of them sold worldwide. 
 * Procesada: Yeah ,COMMA there's <NUM\> million of them sold worldwide .PERIOD 

Esta función se basa en *prepare_text*, del mismo módulo **preprocessor.py**.

In [36]:
from preprocessor import prepare_file

In [37]:
TEST_PREP_PATH  = prepare_file(TEST_RAW_PATH,'test.prepared.txt')
CHECK_PREP_PATH = prepare_file(CHECK_RAW_PATH,'check.prepared.txt',allow_duplicates=False)
TRAIN_PREP_PATH = prepare_file(TRAIN_RAW_PATH,'train.prepared.txt',lowercase=True)

print(TEST_PREP_PATH)
print(CHECK_PREP_PATH)
print(TRAIN_PREP_PATH)

.\data\prepared\test.prepared.txt
.\data\prepared\check.prepared.txt
.\data\prepared\train.prepared.txt


El módulo data.py requiere de varios archivos para poder generar los archivos necesarios para el entrenamiento del modelo. Para ello hemos definido una función 'train_dev_test_split' que separará el archivo de entrenamiento test.prepared.txt en otros tres.

In [38]:
from preprocessor import train_dev_test_split

In [39]:
TRAIN_TRAIN_PREP_PATH, TRAIN_DEV_PREP_PATH, TRAIN_TEST_PREP_PATH = train_dev_test_split(TRAIN_PREP_PATH, train_split = 0.7, dev_split = 0.15)

print(TRAIN_TRAIN_PREP_PATH)
print(TRAIN_DEV_PREP_PATH)
print(TRAIN_TEST_PREP_PATH)

.\data\prepared\train.split.train.txt
.\data\prepared\train.split.dev.txt
.\data\prepared\train.split.test.txt


Con estos archivos ya podemos ejecutar el módulo **data.py** del modelo punctuator2tf2. Para ello debemos pasarle el directorio de los archivos anteriores. Ahí busca aquellos que terminan en *train.txt, *test.txt y *dev.txt.

In [7]:
#!python {settings.PUNCTUATOR2TF2_DIR}/data.py {settings.DATA_PREPARED_DIR}

Vocabulary size: 45087
Vocabulary "data/preprocessed/vocabulary" size: 45087
Vocabulary "data/preprocessed/punctuations" size: 7
0.22% UNK-s in data/preprocessed/train
Vocabulary "data/preprocessed/vocabulary" size: 45087
Vocabulary "data/preprocessed/punctuations" size: 7
0.86% UNK-s in data/preprocessed/dev
Vocabulary "data/preprocessed/vocabulary" size: 45087
Vocabulary "data/preprocessed/punctuations" size: 7
0.84% UNK-s in data/preprocessed/test


Los archivos vectorizados junto a los vocabularios de palabras y puntuación se encuentran en el directorio data/preprocessed.

In [40]:
print('Archivos disponibles en {}: '.format(settings.DATA_PREPROCESSED_DIR),os.listdir(settings.DATA_PREPROCESSED_DIR))

Archivos disponibles en .\data\preprocessed:  ['dev', 'punctuations', 'test', 'train', 'vocabulary']


Ya está todo preparado para entrenar el modelo. Definimos los parámetros:

In [41]:
model_name = 'punctuator'
n_layers = 256
lr = 0.02
model_full_name = 'Model_{}_h{}_lr{}.pcl'.format(model_name,n_layers,lr)
print(model_full_name)

Model_punctuator_h256_lr0.02.pcl


Lo entrenamos (tarda entorno a media hora con varias GPUs de alto rendimiento de Colab):

In [9]:
#!python {settings.PUNCTUATOR2TF2_DIR}/main.py {model_name} {n_layers} {lr}

256 0.02 Model_punctuator_h256_lr0.02.pcl
Building model ...
Vocabulary "data/preprocessed/vocabulary" size: 45087
2022-06-20 05:59:33.918308: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Vocabulary "data/preprocessed/vocabulary" size: 45087
Vocabulary "data/preprocessed/punctuations" size: 7
[TensorShape([45087, 256]), TensorShape([256, 7]), TensorShape([1, 7]), TensorShape([256, 512]), TensorShape([512, 512]), TensorShape([1, 512]), TensorShape([512]), TensorShape([256, 256]), TensorShape([512, 256]), TensorShape([256, 256]), TensorShape([1, 256]), TensorShape([512, 512]), TensorShape([256, 512]), TensorShape([1, 512]), TensorShape([512, 256]), TensorShape([256, 256]), TensorShape([1, 256]), TensorShape([256, 512]), TensorShape([256, 512]), TensorShape([1, 512]), TensorShape([256, 256]), TensorShape([256, 256]), TensorShape([1, 256]), Tensor

Se ha generado el archivo Model_punctuator_h256_lr0.02.pcl en la raíz, con los parámetros del modelo. Ahora vamos a hacer la prediccion del archivo de test. Definimos antes la ruta del archivo puntuado.

In [42]:
output_predicted_file_path = os.path.join(settings.PREDICTED_DIR,'test.{}.raw.txt'.format(model_full_name))
print(output_predicted_file_path)

.\predicted\test.Model_punctuator_h256_lr0.02.pcl.raw.txt


Realizamos la puntuación (tarda entorno a una hora con varias GPUs de alto rendimiento de Colab).

In [8]:
#!python {settings.PUNCTUATOR2TF2_DIR}/punctuator.py ./{model_full_name} {TEST_RAW_PATH} {output_predicted_file_path}

Vocabulary "data/preprocessed/vocabulary" size: 45087
Loading model parameters...
2022-06-20 07:00:08.301113: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Vocabulary "data/preprocessed/vocabulary" size: 45087
Vocabulary "data/preprocessed/punctuations" size: 7
[TensorShape([45087, 256]), TensorShape([256, 7]), TensorShape([1, 7]), TensorShape([256, 512]), TensorShape([512, 512]), TensorShape([1, 512]), TensorShape([512]), TensorShape([256, 256]), TensorShape([512, 256]), TensorShape([256, 256]), TensorShape([1, 256]), TensorShape([512, 512]), TensorShape([256, 512]), TensorShape([1, 512]), TensorShape([512, 256]), TensorShape([256, 256]), TensorShape([1, 256]), TensorShape([256, 512]), TensorShape([256, 512]), TensorShape([1, 512]), TensorShape([256, 256]), TensorShape([256, 256]), TensorShape([1, 256]), TensorShape([256, 512]), TensorShape([2

El archivo anterior es necesario preprarlo para tokenizar los valores numéricos ya que el evaluador de los autores compara palabra a palabra y así nos evitamos errores con signos de puntuación entre caracteres, como por ejemplo numeros decimales.

In [43]:
TEST_PREDICT_PREP_PATH  = prepare_file(output_predicted_file_path,'test.{}.prepared.txt'.format(model_full_name),lowercase=False,allow_duplicates=False,token_punct=False)
print(TEST_PREDICT_PREP_PATH)

.\data\prepared\test.Model_punctuator_h256_lr0.02.pcl.prepared.txt


Procedemos a evaluar el modelo con el módulo **error_calculator.py** de los autores

In [44]:
!python {settings.PUNCTUATOR2TF2_DIR}/error_calculator.py {CHECK_PREP_PATH} {TEST_PREDICT_PREP_PATH}

Vocabulary "data\preprocessed\punctuations" size: 7
----------------------------------------------
PUNCTUATION      PRECISION RECALL    F-SCORE  
?QUESTIONMARK    56.000    26.000    35.500   
.PERIOD          75.200    95.600    84.200   
,COMMA           75.200    64.400    69.400   
:COLON           38.300    36.200    37.200   
!EXCLAMATIONMARK 6.500     5.200     5.700    
;SEMICOLON       27.700    13.600    18.300   
----------------------------------------------
Overall          74.000    75.700    74.800   
Err: 5.19%
SER: 37.6%


Los autores obtuvieron con los mismos parámetros de modelo y para el mismo dataset los siguientes resultados (evaluando sólo los signos ,COMMA ?QUESTIONMARK y .PERIOD):

PUNCTUATION      | PRECISION | RECALL    | F-SCORE
--- | --- | --- | ---
,COMMA           | 64.4 | 45.2 | 53.1
?QUESTIONMARK    | 67.5 | 58.7 | 62.8
.PERIOD          | 72.3 | 71.5 | 71.9
_Overall_        | _68.9_ | _58.1_ | _63.1_

Observamos que obtenemos unas métricas superiores a las que los autores presentan (precision_overall = 74.0, recall_overall = 75,7 y F1_overall = 74.8). Esto se debe principalmente a las modificaciones realizadas en el módulo **punctuator**:
 * Ahora pone la primera letra en mayúscula.
 * Si el último token no es un signo de final de oración se añade un .PERIOD
 * La palabra tras un signo de final de oración ahora se pone en mayúscula.



Ahora vamos a usar la función de evaluación que definimos en la parte 1 basada en la función *VerifyPunctuation*. Para ello importamos versiones de estas funciones que en lugar de hacer la predicción in situ como en el apartado 1, toman como entradas archivos ya puntuados por un modelo.

In [45]:
from evaluator import evaluate_punctuated

In [46]:
evaluate_punctuated(CHECK_PREP_PATH,TEST_RAW_PATH,TEST_PREDICT_PREP_PATH,prepared=True)

MÉTRICAS
precision global:  0.7891876531808333
recall global:  0.6978234241025157
F1 global:  0.7406987598346447
precision media:  0.8370715261265561
recall medio:  0.7736186422895313
F1 medio:  0.8040952260583556
rendimiento:  0.3415380336531776
número de instancias en el corpus:  14382


{'precision_global': 0.7891876531808333,
 'recall_global': 0.6978234241025157,
 'F1_global': 0.7406987598346447,
 'precision_mean': 0.8370715261265561,
 'recall_mean': 0.7736186422895313,
 'F1_mean': 0.8040952260583556,
 'score': 0.3415380336531776}

Las métricas obtenidas por nuestro evaluador son bastante parecidas a las obtenidas por los autores, teniendo en cuenta que el cómputo se hace distinto y ellos las calculan signo a signo. En cualquier caso, el resultado es bastante bueno y mejor que cualquiera de los modelos de la parte 1.

Para finalizar, podemos explorar las distintas predicciones hechas por este modelo usando la versión adaptada a entrada de archivo de *evaluate_from_corpus* del apartado 1.

In [47]:
from evaluator import evaluate_example_from_corpus_punctuated

In [48]:
corpus_line = 88
evaluate_example_from_corpus_punctuated(CHECK_PREP_PATH,TEST_RAW_PATH,TEST_PREDICT_PREP_PATH,corpus_line,prepared=True, readeable=True)

TEST LINE: 
  and the world production today everybody combined making laptops is five million a month so i'm standing here telling you that sometime next year we're going to make 20 percent of the world production
MODEL PUNCTUATED LINE: 
  And the world production today, everybody combined, making laptops. Is five million a month. So i'm standing here telling you that sometime next year, we're going to make <NUM> percent of the world production.
VALIDATION LINE: 
  And the world production today, everybody combined, making laptops, is five million a month. So I'm standing here telling you that sometime next year, we're going to make <NUM> percent of the world production. 

Modificaciones necesarias:  {('I', 5), ('I', 34), ('S', 28), ('I', 24), ('I', 14), ('S', 15), ('I', 7), ('S', 14), ('I', 9), ('S', 0)}
Modificaciones hechas por el modelo:  {('S', 9), ('I', 5), ('I', 34), ('S', 28), ('I', 24), ('I', 14), ('I', 7), ('S', 14), ('I', 9), ('S', 0)}
Diferencias entre modelo y validación: